In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# carrega os arquivos alterando o tipo de dados da data

df = pd.read_csv("../input/train.csv", parse_dates=[0])


In [ ]:
test = pd.read_csv("../input/test.csv", parse_dates=[0])

In [ ]:

df.rename(columns={'count':'rentals'},inplace=True)

In [ ]:
df.head()

In [ ]:
# aquela dica para aproximar os valores e diminuir as diferenças dos erros 
df['rentals'] = np.log(df['rentals'])

In [ ]:
1000 - 1100, 10-11

In [ ]:
import math

In [ ]:
# prova da aproximação usando a função "log"

math.log(1000) - math.log(1100), math.log(10) - math.log(11)

In [ ]:
# outra forma de concatenar arquivos, além do append
df = pd.concat([df,test])

In [ ]:
# outra forma de chamar a coluna é  " df.datetime.dt.year " ou " df['datetime'].dt.year "

df['year'] = df.datetime.dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df.datetime.dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df.datetime.dt.hour

In [ ]:
# separando novamente os dataframes, pois juntamos apenas para facilitar as alterações de dados e criação de colunas

test = df[df['rentals'].isnull()] # retorna apenas os registros com a coluna "rentals" = null

In [ ]:
df = df[~df['rentals'].isnull()] # retorna apenas os que não são nulos (o " ~ " é a negação da condição)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# aqui definimos uma ' semente ', ou seja, a forma como serão selecionadas as colunas para treino e validação
# partindo do dataframe original " df ".
# o número " 42 " é o índice mais utilizado para separar a amostra em treino e validação

train, valid = train_test_split(df, random_state=42)

In [ ]:
# cria uma lista com os nomes das colunas que serão removidas para que o modelo rode adequadamente. A coluna ' rentals ' será o alvo

removed_cols = ['rentals', 'casual', 'registered', 'datetime']

In [ ]:
# cria um novo 
feats = [c for c in df.columns if c not in removed_cols]

In [ ]:
# importa o modelo decision tree, que é muito utilizado

from sklearn.tree import DecisionTreeRegressor

# cria uma instância do modelo, como se fosse um " alias "
dt = DecisionTreeRegressor(random_state=42, max_depth=2)

In [ ]:
# utilizar o comando " fit " para treinar o modelo. Neste caso utilizamos a coluna ' rentals ' para treinar e prever quantas locações terão

dt.fit(train[feats], train['rentals'])


In [ ]:
# importa uma biblioteca para apresentação visual da árvore de decisão

from fastai.structured import draw_tree

In [ ]:
# executa a função para mostrar o resultado do modelo de forma visual

draw_tree(dt, train[feats], precision=3, size=40)

In [ ]:
# o campo " value " é o valor do número de locações 
# - lembrando que estão com a função log e abaixo temos a previsão real de locações geradas pelo modelo
math.exp(2.78), math.exp(5.27)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# cria essa coluna apenas para simplificar a validação do modelo com uma previsão, que foi uma constante (média das quantidades de locações)
train['preds'] = train['rentals'].mean()

In [ ]:
# tentando validar / explicar os resultados do modelo " decisiontree "

train.shape, train['rentals'].mean()

In [ ]:
# 1o parâmetro é igual ao que realmente aconteceu, e o segundo é um cálculo do erro quadrado do real com a média de toda a amostra
mean_squared_error(train['rentals'], train['preds'])

In [ ]:
# condição criada para o segundo grupo (para ' hour ' <= 6.5 ) - valor que está igual ao " value " do segundo quadrado da árvore
train[train['hour']<6.5]['rentals'].mean()

In [ ]:
# vamos deletar a coluna " preds ", pois foi criada apenas para validarmos/explicarmos o resultado do modelo
del train['preds']

In [ ]:
# importa a randomforest

from sklearn.ensemble import RandomForestRegressor


In [ ]:
# criando um "alias" para o meu modelo. O parâmenteo ' n_jobs = -1 ' é para utilizar a quantidade máxima da capacidade do processador

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

In [ ]:
rf.fit(train[feats], train['rentals'])

In [ ]:
# gera uma lista com as previsões para cada coluna da lista " feats "
train_preds = rf.predict(train[feats])

In [ ]:
train_preds

In [ ]:
# pega o erro e eleva ao quadrado (base de treinamento)

mean_squared_error(train['rentals'],train_preds)**(1/2)

In [ ]:
# para validar o modelo, temos que necessariamente utilizar a base de validação
# segue o mesmo exemplo acima e pega o erro e eleva ao quadrado (base de validação)

valid_preds = rf.predict(valid[feats])

mean_squared_error(valid['rentals'],valid_preds)**(1/2)

In [ ]:
test.head()

In [ ]:
# abaixo enviaremos o modelo para o kaggle validar com uma vase de teste que não conhecemos

test['count'] = np.exp(rf.predict(test[feats]))



In [ ]:
test[['datetime','count']].to_csv('rf.csv', index=False)